<a href="https://colab.research.google.com/github/duybluemind1988/Data-science/blob/master/Sonion/Sonion_process_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot
import re

from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
import scipy.stats as scs
from math import sqrt
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
path='/content/drive/My Drive/Data/Sonion/Control plan and stast/Control Plan data REC 3000 - WK34 - 2020.xlsm'

In [3]:
import xlrd
xls = xlrd.open_workbook(path, on_demand=True)
sheet_names=xls.sheet_names()
print(xls.sheet_names())

['Master list', '39682', '38682', '30682', '37682', '30665', '30660-37660-38660', '30380-37380-38380', '37670-39670', '37662', '30650-37650 (3500T)', '37650-38655-39650', '37641', '30470-37470', '38546', '39615', '38940', '38594', '37560', '30740', '37640-38635', '30674-37674-38674 ', '30672-37672-38672', '30592', '30544-37544-38544-39544', '30538', '30530-37530-39530', '30480-37480-38481', '30370-37370-38370 ', '30750-30940-39750 _S1', '30750-30940-39750 _S2', '37750-37940_S1', '37750-37940_S2', '37751']


In [4]:
sheet_dict={}
xls = pd.ExcelFile(path)
for name in sheet_names:
  sheet_dict[name] = pd.read_excel(xls, name)

In [5]:
name='38682'
sheet=sheet_dict[name]

In [6]:
#sheet.to_csv('abc.xlsx')

In [7]:
# get information in each coil
#dim_name=sheet[sheet.columns[14]][22]
#max=sheet[sheet.columns[14]][23]
#min=sheet[sheet.columns[14]][24]
#date=pd.to_datetime(sheet[sheet.columns[9]][25:])
#dim_value=sheet[sheet.columns[14]][25:]

In [8]:
# find begin and end col
values_col=sheet.iloc[22,:]
values_col.reset_index(drop=True,inplace=True)
begin_col=values_col[values_col=='Kích thước\nDimension'].index
begin_col=begin_col+1
begin_col

Int64Index([14], dtype='int64')

In [10]:
end_col=values_col[values_col=='MSNV'].index
end_col

Int64Index([20], dtype='int64')

In [16]:
sheet.columns[begin_col[0]:end_col[0]]

Index(['Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19'],
      dtype='object')

In [64]:
dim_dict={}
tolerance_dict={}
for name in sheet.columns[begin_col[0]:end_col[0]]:
  df=pd.DataFrame()
  
  #tolerance_dict[sheet[name][22]]=[sheet[name][24],sheet[name][23]]
  df['Date']=pd.to_datetime(sheet[sheet.columns[9]][25:])
  df['Value']=sheet[name][25:]
  df['USL']=sheet[name][23] # max
  df['LSL']=sheet[name][24] # min
  df.dropna(subset=['Value'],inplace=True)
  #UCL,LCL,nominal:
  sigma=3
  #df_final = df.assign(
  #              UCL=df['Value'].mean() + df['Value'].std()*sigma,
   #             LCL=df['Value'].mean() - df['Value'].std()*sigma,
    #            mean=df['Value'].mean())
  df['UCL']=df['Value'].mean() + df['Value'].std()*sigma
  df['LCL']=df['Value'].mean() - df['Value'].std()*sigma
  df['Mean']=df['Value'].mean()
  df[df.columns[1:]]=df[df.columns[1:]].astype('float32')
  dim_name=sheet[name][22]
  dim_dict[dim_name]=df.reset_index(drop=True)